<a href="https://colab.research.google.com/github/kooll/mt2024/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


これはテストです。 chapter12 の作業中です。

Think Python 3e*の印刷版と電子書籍版は、[Bookshop.org](https://bookshop.org/a/98697/9781098155438)や[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325)から注文できます。

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

# テキスト解析と生成

この時点で、Python の主要なデータ構造（リスト、辞書、タプル）と、それを使用するいくつかのアルゴリズムをカバーしました。
この章では、それらを用いてテキスト解析とマルコフ生成を探求します:

* テキスト解析は、文書中の単語間の統計的な関係を説明する方法で、例えばある単語の後に他の単語が続く確率などが含まれます。

* マルコフ生成は、元のテキストに似た単語やフレーズを用いて新しいテキストを生成する方法です。

これらのアルゴリズムは、チャットボットの主要な構成要素である大規模言語モデル（LLM）の一部と類似しています。

まず、本の中で各単語が出現する回数を数えることから始めます。
次に、単語のペアを見て、それぞれの単語に続く可能性のある単語のリストを作成します。
簡単なマルコフ生成器のバージョンを作成し、練習として、より一般的なバージョンを作成する機会を持つようにします。

ユニークな単語

テキスト分析の第一歩として、本を読みましょう。ロバート・ルイス・スティーブンソンの『ジキル博士とハイド氏の奇妙な事件』です。そして、ユニークな単語の数を数えましょう。この章のノートブックに本のダウンロード手順があります。

次のセルは、Project Gutenbergから本をダウンロードします。

In [ ]:
download('https://www.gutenberg.org/cache/epub/43/pg43.txt');

プロジェクト・グーテンベルクから入手できるバージョンには、冒頭に書籍情報が含まれており、最後にライセンス情報があります。これらの部分を削除するために、第8章で説明されている`clean_file`を使用して、書籍のテキストのみを含む「クリーン」ファイルを書きます。

In [ ]:
def is_special_line(line):
    return line.strip().startswith('*** ')

In [ ]:
def clean_file(input_file, output_file):
    reader = open(input_file, encoding='utf-8')
    writer = open(output_file, 'w')

    for line in reader:
        if is_special_line(line):
            break

    for line in reader:
        if is_special_line(line):
            break
        writer.write(line)

    reader.close()
    writer.close()

In [ ]:
filename = 'dr_jekyll.txt'

In [ ]:
clean_file('pg43.txt', filename)

`for`ループを使用してファイルから行を読み込み、`split`を使って行を単語に分割します。次に、ユニークな単語を管理するために、各単語を辞書のキーとして保存します。

In [ ]:
unique_words = {}
for line in open(filename):
    seq = line.split()
    for word in seq:
        unique_words[word] = 1

len(unique_words)

辞書の長さはユニークな単語の数で、これを数える方法では約「6000」あります。
しかし、それらを調べると、一部が有効な単語でないことがわかります。

例えば、「unique_words」の中で最も長い単語を見てみましょう。
単語を長さ順にソートするために、`sorted`を使用し、`len`関数をキーワード引数として渡します。

In [ ]:
sorted(unique_words, key=len)[-5:]

スライスインデックス `[-5:]` は、ソートされたリストの最後の `5` 要素を選択し、これらが最も長い単語です。

リストには「circumscription」などの本当に長い単語や、「chocolate-coloured」のようなハイフンでつながれた単語も含まれています。しかし、最も長い「単語」の中には、実際にはダッシュで区切られた2つの単語があります。また、句読点としてピリオド、感嘆符、引用符などが含まれている単語もあります。

それでは、次に進む前に、ダッシュやその他の句読点を処理しましょう。

## 句読点

テキスト内の単語を識別するためには、次の2つの問題に対処する必要があります。

* 行内にダッシュが現れた場合、それをスペースに置き換える必要があります。これにより、`split`を使用した際に単語が分離されます。

* 単語を分割した後、`strip`を使用して句読点を削除できます。

最初の問題に対処するためには、次のような関数を使用できます。この関数は文字列を取り込み、ダッシュをスペースに置き換え、文字列を分割し、結果のリストを返します。

In [ ]:
def split_line(line):
    return line.replace('—', ' ').split()

「split_line」はダッシュのみを置換し、ハイフンには対応しないことに注意してください。こちらがその例です。

In [ ]:
split_line('coolness—frightened')

さて、各単語の先頭と末尾から句読点を取り除くには、`strip`メソッドを使用できますが、句読点と見なされる文字のリストが必要です。

Pythonの文字列内の文字はUnicodeで表されており、これはほぼすべてのアルファベットの文字、数字、記号、句読点などを表すための国際標準です。`unicodedata`モジュールは、文字がどのカテゴリに属しているかを教えてくれる`category`関数を提供しています。文字を入力すると、その文字がどのカテゴリに属しているかを示す文字列を返します。

In [ ]:
import unicodedata

unicodedata.category('A')

`'A'` のカテゴリ文字列は `'Lu'` です。このうち `'L'` は文字であることを意味し、`'u'` は大文字であることを意味します。

`'.'` のカテゴリ文字列は `'Po'` です。このうち `'P'` は句読点であることを意味し、`'o'` はサブカテゴリが「その他」であることを意味します。

In [ ]:
unicodedata.category('.')

本の中の句読点は、カテゴリが `'P'` で始まる文字を探すことで見つけることができます。以下のループでは、一意の句読点を辞書に保存します。

In [ ]:
punc_marks = {}
for line in open(filename):
    for char in line:
        category = unicodedata.category(char)
        if category.startswith('P'):
            punc_marks[char] = 1

句読点のリストを作成するには、辞書のキーを文字列として結合することができます。

In [ ]:
punctuation = ''.join(punc_marks)
print(punctuation)

本の中でどの文字が句読点であるかを知ったので、単語を受け取り、前後の句読点を取り除き、小文字に変換する関数を書くことができます。

In [ ]:
def clean_word(word):
    return word.strip(punctuation).lower()

こちらが例です。

In [ ]:
clean_word('“Behold!”')

`strip` は文字列の先頭と末尾の文字を削除するため、ハイフンでつながれた単語はそのまま残ります。

In [ ]:
clean_word('pocket-handkerchief')

こちらが、`split_line`と`clean_word`を使用して、書籍内のユニークな単語を特定するループです。

In [ ]:
unique_words2 = {}
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        unique_words2[word] = 1

len(unique_words2)

この厳密な言葉の定義によると、約4000のユニークな単語があります。そして、最長の単語のリストが整理されたことを確認できます。

In [ ]:
sorted(unique_words2, key=len)[-5:]

では、各単語が何回使われているか見てみましょう。

## 単語の出現頻度

次のループは、各ユニークな単語の出現頻度を計算します。

In [ ]:
word_counter = {}
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1

初めて単語を見たとき、その頻度を「1」に初期化します。その後、同じ単語を再び見かけた場合、頻度を増やします。

どの単語が最も頻繁に現れるかを確認するために、`items`メソッドを使って`word_counter`からキーと値のペアを取得し、それらをペアの2番目の要素、つまり頻度でソートします。
まず、2番目の要素を選択する関数を定義しましょう。

In [ ]:
def second_element(t):
    return t[1]

`sorted` を2つのキーワード引数と共に使用できます：

* `key=second_element` は、語の頻度に基づいてアイテムがソートされることを意味します。

* `reverse=True` は、アイテムが逆順にソートされ、最も頻度の高い語が最初に来ることを意味します。

In [ ]:
items = sorted(word_counter.items(), key=second_element, reverse=True)

こちらが最も頻繁に使われる5つの単語です。

In [ ]:
for word, freq in items[:5]:
    print(freq, word, sep='\t')

次のセクションでは、このループを関数にカプセル化します。そして、それを用いて新機能である「オプションのパラメータ」を示します。

## オプションのパラメータ

これまで、オプションのパラメータを取る組み込み関数を使用してきました。例えば、`round` 関数は、保持する小数点以下の桁数を示す `ndigits` というオプションのパラメータを受け付けます。

In [ ]:
round(3.141592653589793, ndigits=3)

しかし、組み込み関数だけではなく、オプションのパラメータを持つ関数も自分で書くことができます。例えば、次の関数は `word_counter` と `num` という2つのパラメータを取ります。

In [ ]:
def print_most_common(word_counter, num=5):
    items = sorted(word_counter.items(), key=second_element, reverse=True)

    for word, freq in items[:num]:
        print(freq, word, sep='\t')

2番目のパラメーターは代入文のように見えますが、そうではありません。これはオプションのパラメーターです。

この関数を1つの引数で呼び出すと、`num`は**デフォルト値**である`5`を取得します。

In [ ]:
print_most_common(word_counter)

この関数を2つの引数で呼び出すと、2番目の引数がデフォルト値の代わりに `num` に割り当てられます。

In [ ]:
print_most_common(word_counter, 3)

その場合、オプションの引数がデフォルト値を**オーバーライド**すると言います。

関数に必須パラメーターとオプションパラメーターが両方ある場合、すべての必須パラメーターが最初に来て、その後にオプションのものが続く必要があります。

In [ ]:
%%expect SyntaxError

def bad_function(n=5, word_counter):
    return None

本をスペルチェックする必要があるとしましょう。つまり、誤って綴られた可能性のある単語を見つけ出したいということです。その方法の一つとして、有効な単語のリストに現れない単語を本の中から見つけ出すことが考えられます。以前の章では、スクラブルのようなワードゲームで有効とされる単語のリストを使用しました。今回は、このリストを使ってロバート・ルイス・スティーブンソンの著作をスペルチェックします。

この問題を集合の差として考えることができます。すなわち、ある集合（本の中の単語）から別の集合（リストの中の単語）に含まれないすべての単語を見つけたいということです。

次のセルは単語リストをダウンロードします。

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

以前行ったように、`words.txt` の内容を読み取り、それを文字列のリストに分割することができます。

In [ ]:
word_list = open('words.txt').read().split()

次に、単語を辞書のキーとして保存し、それを使って`in`演算子でその単語が有効かどうかを素早く確認できるようにします。

In [ ]:
valid_words = {}
for word in word_list:
    valid_words[word] = 1

さて、本に出てくる単語で単語リストにないものを特定するために、2つの辞書をパラメーターとして受け取り、一方にあって他方にないすべてのキーを含む新しい辞書を返す`subtract`を使用します。

In [ ]:
def subtract(d1, d2):
    res = {}
    for key in d1:
        if key not in d2:
            res[key] = d1[key]
    return res

こちらがその使い方です。

In [ ]:
diff = subtract(word_counter, valid_words)

単語のサンプルを抽出するために、`diff`で最も一般的な単語を印刷することができます。

In [ ]:
print_most_common(diff)

最も一般的な「スペルミス」の単語は、主に名前と、いくつかの一文字の単語です（ミスター・アターソンはドクター・ジーキルの友人であり弁護士です）。

もし一度だけ出現する単語を選ぶと、それらは実際のスペルミスである可能性が高くなります。それを行うために、項目をループして、頻度が「1」の単語のリストを作成することができます。

In [ ]:
singletons = []
for word, freq in diff.items():
    if freq == 1:
        singletons.append(word)

こちらがリストの最後のいくつかの要素です。

In [ ]:
singletons[-5:]

それらのほとんどは単語リストにない有効な単語です。しかし、「reindue」は「reinduce」の誤字のようなので、少なくとも1つの正当なエラーを見つけました。

## ランダムな数

マルコフテキスト生成へのステップとして、次に`word_counter`からランダムな単語のシーケンスを選んでみます。
しかしその前に、ランダム性について話しましょう。

同じ入力を与えられたとき、ほとんどのコンピュータプログラムは**決定論的**であり、つまり毎回同じ出力を生成します。
決定論は通常は良いことです。なぜなら、同じ計算が同じ結果をもたらすことを期待するからです。
しかし、いくつかのアプリケーションでは、コンピュータが予測不可能であってほしいこともあります。
例としてはゲームがありますが、他にもあります。

プログラムを本当に非決定的なものにするのは難しいのですが、それを偽装する方法はいくつかあります。
その一つが、**疑似乱数**を生成するアルゴリズムを使用することです。
疑似乱数は決定論的な計算によって生成されているため、真にランダムではありません。
しかし、それらの数列を見るだけでは、ランダムなものと区別するのはほぼ不可能です。

`random`モジュールは疑似乱数を生成する関数を提供しています。ここからはそれを単に「乱数」と呼びます。
このようにしてインポートできます。

In [ ]:
import random

In [ ]:
# this cell initializes the random number generator so it
# generates the same sequence each time the notebook runs.

random.seed(4)

`random`モジュールには`choice`という関数があり、これはリストからランダムに要素を選びます。すべての要素が選ばれる確率は同じです。

In [ ]:
t = [1, 2, 3]
random.choice(t)

関数を再度呼び出すと、同じ要素が再び返されるか、異なる要素が返される可能性があります。

In [ ]:
random.choice(t)

長期的には、すべての要素がほぼ同じ回数得られることを期待しています。

`choice`を辞書と一緒に使用すると、`KeyError`が発生します。

In [ ]:
%%expect KeyError

random.choice(word_counter)

ランダムなキーを選ぶには、キーをリストに入れてから `choice` 関数を呼び出す必要があります。

In [ ]:
words = list(word_counter)
random.choice(words)

ランダムな単語の列を生成しても、あまり意味がありません。

In [ ]:
for i in range(6):
    word = random.choice(words)
    print(word, end=' ')

問題の一部は、ある単語が他の単語よりも一般的であることを考慮していないことです。異なる「重み」を持つ単語を選ぶと、結果はより良くなります。そのため、ある単語は他の単語よりも頻繁に選ばれることになります。

`word_counter`の値を重みとして使用すると、各単語はその頻度に応じた確率で選ばれます。

In [ ]:
weights = word_counter.values()

`random`モジュールには、オプションの引数として重みを取ることができる`choices`という別の関数があります。

In [ ]:
random.choices(words, weights=weights)

さらに、選択する単語の数を指定するオプション引数 `k` も取ります。

In [ ]:
random_words = random.choices(words, weights=weights, k=6)
random_words

結果は文字列のリストであり、それを結合してより文章らしくすることができます。

In [ ]:
' '.join(random_words)

本からランダムに単語を選ぶと、その語彙の感覚を得ることはできますが、一連のランダムな単語が意味を成すことはまれです。というのも、連続する単語の間に関係がないからです。たとえば、実際の文の中では、「the」のような冠詞の後には形容詞や名詞が続くことを期待し、動詞や副詞が続く可能性は低いです。したがって、次のステップは単語間のこれらの関係性を調べることです。

## バイグラム

1 つの単語を見るのではなく、2 つの単語のシーケンス、すなわち**バイグラム**を見ていきます。
3 つの単語のシーケンスは**トライグラム**と呼ばれ、不特定の数の単語のシーケンスは**n-グラム**と呼ばれます。

本に出てくるすべてのバイグラムを見つけ、それぞれの出現回数を数えるプログラムを書いてみましょう。
結果を保存するために、以下の要件を満たす辞書を使用します。

* キーはバイグラムを表す文字列のタプルで、

* 値は出現頻度を表す整数です。

これを `bigram_counter` と呼びましょう。

In [ ]:
bigram_counter = {}

次の関数は、2つの文字列を要素に持つリストをパラメータとして受け取ります。まず、この2つの文字列をタプルに変換し、それを辞書でキーとして使用できるようにします。その後、そのキーを `bigram_counter` に追加します。キーが存在しない場合は新たに作成し、すでに存在する場合は頻度を増加させます。

In [ ]:
def count_bigram(bigram):
    key = tuple(bigram)
    if key not in bigram_counter:
        bigram_counter[key] = 1
    else:
        bigram_counter[key] += 1

本を読み進める際、すべての連続した単語のペアを追跡する必要があります。  
例えば、"man is not truly one" という順序が表示された場合には、`man is`、`is not`、`not truly` などのバイグラムを追加します。

これらのバイグラムを追跡するために、`window` と呼ばれるリストを使用します。これは、書籍のページの上をスライドして、常に2つの単語しか表示しない窓のようなものです。  
最初は、`window` は空です。

In [ ]:
window = []

それぞれの単語を一度に一つずつ処理するために、以下の関数を使用します。

In [ ]:
def process_word(word):
    window.append(word)

    if len(window) == 2:
        count_bigram(window)
        window.pop(0)

この関数が初めて呼び出されたとき、指定された単語を`window`に追加します。`window`にはまだ一つの単語しかないため、ビグラムは存在せず、関数は終了します。

二回目以降に呼び出されるときは、`window`に二つ目の単語を追加します。`window`に二つの単語があるので、`count_bigram`を呼び出して、各ビグラムが出現する回数を記録します。その後、`pop`を使って`window`から最初の単語を削除します。

次のプログラムは本の中の単語を順番にループし、1つずつ処理します。

In [ ]:
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word(word)

この結果は、各バイグラムが出現する回数をマッピングする辞書です。最も一般的なバイグラムを確認するには、`print_most_common`を使用できます。

In [ ]:
print_most_common(bigram_counter)

これらの結果を見ると、どの単語のペアが一緒に現れる可能性が高いかを把握することができます。また、結果を利用してこのようなランダムな文を生成することもできます。

In [ ]:
random.seed(0)

In [ ]:
bigrams = list(bigram_counter)
weights = bigram_counter.values()
random_bigrams = random.choices(bigrams, weights=weights, k=6)

「bigrams」は、書籍に現れるバイグラム（2つの連続する単語の組み合わせ）のリストです。「weights」はそれらの頻度のリストであり、したがって「random_bigrams」は、バイグラムが選択される確率がその頻度に比例するサンプルです。

以下がその結果です。

In [ ]:
for pair in random_bigrams:
    print(' '.join(pair), end=' ')

この方法でテキストを生成する方が単語をランダムに選ぶよりは良いですが、それでもあまり意味をなさないことがあります。

マルコフ解析

マルコフ連鎖のテキスト解析を使用すると、テキスト内の各単語に対して、次に続く単語のリストを計算することができます。例として、モンティ・パイソンの「Eric, the Half a Bee」の歌詞を分析します。

In [ ]:
song = """
Half a bee, philosophically,
Must, ipso facto, half not be.
But half the bee has got to be
Vis a vis, its entity. D'you see?
"""

結果を保存するために、各単語からそれに続く単語のリストへのマッピングを行う辞書を使用します。

In [ ]:
successor_map = {}

例として、その曲の最初の2つの単語から始めてみましょう。

In [ ]:
first = 'half'
second = 'a'

最初の単語が `successor_map` に存在しない場合、最初の単語から2番目の単語を含むリストへのマッピングを追加する新しい項目を追加する必要があります。

In [ ]:
successor_map[first] = [second]
successor_map

最初の単語がすでに辞書にある場合は、これまでに見た後続のリストを調べて、新しいものを追加することができます。

In [ ]:
first = 'half'
second = 'not'

successor_map[first].append(second)
successor_map

次の関数は、これらのステップをカプセル化します。

In [ ]:
def add_bigram(bigram):
    first, second = bigram

    if first not in successor_map:
        successor_map[first] = [second]
    else:
        successor_map[first].append(second)

同じバイグラムが複数回登場する場合、2番目の単語がリストに複数回追加されます。
このようにして、`successor_map` は各後続単語がどれだけ出現するかを記録します。

前のセクションで行ったように、連続する単語のペアを格納するために `window` というリストを使用します。
そして、以下の関数を用いて単語を1つずつ処理します。

In [ ]:
def process_word_bigram(word):
    window.append(word)

    if len(window) == 2:
        add_bigram(window)
        window.pop(0)

この方法で曲の歌詞を処理します。

In [ ]:
successor_map = {}
window = []

for word in song.split():
    word = clean_word(word)
    process_word_bigram(word)

そして、こちらが結果です。

In [ ]:
successor_map

さて、本を分析しましょう。

In [ ]:
successor_map = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_bigram(word)

私たちはどんな単語でも調べて、その後に続く単語を見つけることができます。

In [ ]:
# I used this cell to find a predecessor with a good number of possible successors
# and at least one repeated word.

def has_duplicates(t):
    return len(set(t)) < len(t)

for key, value in successor_map.items():
    if len(value) == 7 and has_duplicates(value):
        print(key, value)

In [ ]:
successor_map['going']

この後継者のリストでは、単語「'to'」が3回出現していることに注意してください。他の後継者は1回しか出現していません。

## 生成テキスト

前のセクションの結果を用いて、オリジナルと同じ連続する単語間の関係を持つ新しいテキストを生成できます。以下はその手順です：

* テキスト内に現れる任意の単語から始め、その単語の後に続く可能性のある単語を調べ、ランダムに1つ選びます。

* 次に、選ばれた単語を使って、その単語の後に続く可能性のある単語を調べ、ランダムに1つ選びます。

このプロセスを繰り返して、希望するだけ多くの単語を生成できます。
例として、単語「'although'」から始めてみましょう。
これに続くことができる単語は以下の通りです。

In [ ]:
word = 'although'
successors = successor_map[word]
successors

In [ ]:
# this cell initializes the random number generator so it
# starts at the same point in the sequence each time this
# notebook runs.

random.seed(2)

`choice`を使用して、リストから等しい確率で選ぶことができます。

In [ ]:
word = random.choice(successors)
word

同じ単語がリストに複数回現れる場合、その単語が選ばれる可能性が高くなります。

これらのステップを繰り返すことで、次のループを使用してより長いシリーズを生成できます。

In [ ]:
for i in range(10):
    successors = successor_map[word]
    word = random.choice(successors)
    print(word, end=' ')

結果はより本物らしい文章に聞こえるが、それでもあまり意味を成していません。

`successor_map`で一つ以上の単語をキーとして使用することで、より良い結果を望むことができます。
たとえば、各バイグラムやトライグラムから次に来る単語のリストをマップする辞書を作成することができます。
練習として、この分析を実装し、その結果がどのように見えるか確認する機会があるでしょう。

## デバッグ

この段階で、より大規模なプログラムを書いており、デバッグに多くの時間を費やしているかもしれません。難しいバグで行き詰まった場合は、次のことを試してみてください。

* 読む: コードを確認し、自分自身に読み聞かせながら、それがあなたの意図したことを正しく表現しているかを確認します。

* 実行: 変更を加えて異なるバージョンを実行して実験します。プログラムの適切な場所で正しいものを表示すれば、問題が明らかになることがありますが、時には足場を組み立てる必要があります。

* 考える: 時間を取って考えます！それはどんなエラーですか: 構文エラー、実行時エラー、セマンティックエラーのいずれですか？エラーメッセージやプログラムの出力からどんな情報が得られますか？あなたが目にしている問題を引き起こす可能性のあるエラーは、どんなものでしょうか？問題が発生する前に最後に何を変更しましたか？

* ラバーダッキング: 問題を他の人に説明することで、質問を終える前に答えが見つかることがあります。多くの場合、他の人は必要ありません。ラバーダックに話しかけるだけでいいのです。そしてこれが、よく知られた戦略である「ラバーダック・デバッグ」の由来です。本当の話です -- 詳しくは <https://en.wikipedia.org/wiki/Rubber_duck_debugging> を参照してください。

* 後退: ある時点で、最善の策はバックアップすることです -- 最近の変更を元に戻すことです。それで動くプログラムに戻れたら、そこから再構築を始めることができます。

* 休憩: 脳に休憩を与えると、時々問題の解決策が自然に見つかることがあります。

初心者のプログラマーは、これらの活動のうちの一つにこだわってしまい、他の活動を忘れてしまうことがあります。各活動には独自の失敗モードがあります。

例えば、コードを読むことは、問題がタイプミスである場合には効果的ですが、概念的な誤解が原因である場合には効果がありません。
プログラムの動作を理解していない場合、100回読んでもエラーが見つからないことがあります。エラーはあなたの頭の中にあるからです。

小さくシンプルなテストを実行することで、実験を行うことは有効です。
しかし、考えずにコードを読まずに実験を行うと、何が起こっているのかを把握するのに長い時間がかかることがあります。

時間をかけて考える必要があります。デバッグは実験科学のようなものです。問題が何であるかについて、少なくとも一つの仮説を持つべきです。可能性が二つ以上ある場合、一つを排除できるテストを考えてみてください。

しかし、どんなに優れたデバッグ手法でも、エラーが多すぎたり、修正しようとしているコードが大きくて複雑すぎたりすると、効果を発揮しません。時には、プログラムを簡素化して、動作する状態に戻すのが最良の選択肢です。

初心者のプログラマーは後退することをためらうことが多いです。間違っているとしても、コードの一行を削除することに耐えられないからです。安心するために、コードを削減する前にプログラムを別のファイルにコピーしておくと良いでしょう。そうすれば、部分的に元のコードを一行ずつ戻すことができます。

難しいバグを見つけるには、コードを読む、実行する、熟考する、後退する、そして時には休息することが必要です。この活動のどれかで行き詰まったら、他の方法を試してみてください。

## 用語集

**デフォルト値:**
引数が提供されない場合にパラメーターに割り当てられる値。

**オーバーライド:**
デフォルト値を引数で置き換えること。

**決定論的:**
決定論的なプログラムは、同じ入力が与えられた場合、実行するたびに同じことを行う。

**疑似乱数:**
疑似乱数数列はランダムに見えるが、決定論的なプログラムによって生成される。

**バイグラム:**
通常、単語の2つの要素からなるシーケンス。

**トライグラム:**
3つの要素からなるシーケンス。

**n-グラム:**
指定されていない数の要素からなるシーケンス。

**ラバーダックデバッグ:**
無生物に対して問題を声に出して説明することによって行うデバッグ方法。

## 演習

In [ ]:
# This cell tells Jupyter to provide detailed debugging information
# when a runtime error occurs. Run it before working on the exercises.

%xmode Verbose

バーチャルアシスタントに質問

`add_bigram`において、`if`文はキーが辞書に既に存在するかどうかに応じて、新しいリストを作成するか既存のリストに要素を追加します。

In [ ]:
def add_bigram(bigram):
    first, second = bigram

    if first not in successor_map:
        successor_map[first] = [second]
    else:
        successor_map[first].append(second)

辞書には `setdefault` というメソッドがあり、これを使用して、より簡潔に同じことを行うことができます。
バーチャルアシスタントに、このメソッドの動作について尋ねることができます。または、`add_bigram` をバーチャルアシスタントにコピーし、"`setdefault` を使ってこれを書き直せますか？" と尋ねることも可能です。

この章では、マルコフ連鎖のテキスト解析と生成を実装しました。
興味があれば、このトピックについてもっと詳しく知りたい場合は、バーチャルアシスタントに質問できます。
学べることの一つとして、バーチャルアシスタントがどのようにして多くのアルゴリズムを使用しているかが挙げられます。それらは多くの面で似ていますが、重要な面で異なっています。
バーチャルアシスタントに、「GPTのような大規模言語モデルとマルコフ連鎖テキスト分析の違いは何ですか？」と尋ねてください。

### 演習

3つの単語の連続である各トライグラムが現れる回数を数える関数を書きなさい。
テキストとして『ジキル博士とハイド氏』を用いた場合、最も一般的なトライグラムは「said the lawyer」であることがわかるはずです。

ヒント: `count_bigram`に似た`count_trigram`という関数を書きます。それから`process_word_bigram`に似た`process_word_trigram`という関数を書きます。

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

本を読み取り、単語を処理するには、次のループを使用できます。

In [ ]:
trigram_counter = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

Then use `print_most_common`を使用して、その本の中で最も一般的なトライグラムを見つけます。

In [ ]:
print_most_common(trigram_counter)

申し訳ありませんが、コードの実装について直接の提供はできませんが、関数 `add_bigram` の考え方を `add_trigram` に応用する際のアドバイスを提供します。

1. `add_bigram` 関数の基本的な役割は、与えられた2つの単語のペアをキーとして、3番目の単語をそのペアに続く可能性のある単語としてマップに追加または更新することです。

2. `add_trigram` 関数の場合、3つの単語のうち最初の2つをキーとし、それに続く3番目の単語をそのペアに続く可能性のある単語として `successor_map` に追加または更新する必要があります。

具体的には、以下のような手順で進めることを考えてください：
- `add_trigram` の最初の2つの単語からなるペアをキーとして作成します。
- そのペアが既に `successor_map` に存在するかを確認します。
- 存在する場合は、3番目の単語をリストに追加します。
- 存在しない場合は、新たにそのペアをキーとするエントリを `successor_map` に追加し、そのリストに3番目の単語を含めます。

この方法で、`successor_map` は各2単語のペアに対する3番目の単語の可能性を効率的に管理できます。これを念頭に置いて、コードを構築してください。

In [ ]:
# Solution goes here

`process_word_trigram` 関数は、3-gram（3つの連続する単語の集合）を処理して、それらを保存または扱うために `add_trigram` 関数を呼び出します。以下に、`add_trigram` を呼び出すことで 3-gram を処理する `process_word_trigram` の実装例を示します。

```python
def process_word_trigram(word_list):
    """
    Processes a list of words and calls add_trigram for each trigram found.

    Args:
    word_list: List of words (strings).
    """
    # Ensure there are at least three words to form a trigram
    if len(word_list) < 3:
        return

    for i in range(len(word_list) - 2):  # Loop until the third-last word
        # Extract the trigram
        trigram = (word_list[i], word_list[i + 1], word_list[i + 2])
        
        # Call the add_trigram function with the extracted trigram
        add_trigram(trigram)

def add_trigram(trigram):
    """
    Adds the given trigram to the trigram model or data store.

    Args:
    trigram: Tuple of three words (string).
    """
    # Here you can add code to store or process the trigram
    print(f"Trigram added: {trigram}")
```

この例では、`process_word_trigram` は与えられた単語のリストからトライグラムを抽出し、それを `add_trigram` に渡します。`add_trigram` は現在のところトライグラムを単に表示しますが、この関数にはトライグラムをデータストアに追加したり、他の処理を行うロジックを実装することができます。

In [ ]:
def process_word_trigram(word):
    window.append(word)

    if len(window) == 3:
        add_trigram(window)
        window.pop(0)

以下のループを使用して、「Eric, the Half a Bee」の歌詞であなたの関数をテストすることができます。

In [ ]:
successor_map = {}
window = []

for string in song.split():
    word = string.strip(punctuation).lower()
    process_word_trigram(word)

もしあなたの関数が意図した通りに動作する場合、前置詞 `('half', 'a')` は単一の要素 `'bee'` を持つリストにマッピングされるべきです。実際、この曲の各バイグラムは一度しか登場しないため、`successor_map` のすべての値は単一の要素になります。

In [ ]:
successor_map

次のループを使用して、本の単語で関数をテストできます。

In [ ]:
successor_map = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

次の演習では、結果を使用して新しいランダムなテキストを生成します。

### 練習問題

この練習では、`successor_map` がビグラムごとに続く単語のリストをマッピングする辞書であると仮定します。

In [ ]:
# this cell initializes the random number generator so it
# starts at the same point in the sequence each time this
# notebook runs.

random.seed(3)

ランダムなテキストを生成するために、まず `successor_map` からランダムなキーを選びます。

In [ ]:
successors = list(successor_map)
bigram = random.choice(successors)
bigram

残念ながら、与えられた情報だけでは実際のプログラムコードを提供することはできません。ただし、以下の擬似コードはこの手順の動作を説明します。

```python
import random

# 仮の`successor_map`辞書。実際のデータに置き換えてください。
successor_map = {
    ('doubted', 'if'): ['from', 'that', 'he'],
    ('if', 'from'): ['he', 'she', 'they'],
    # 他のビグラムとその後続のマッピングが続く
}

# 開始するビグラムを設定
bigram = ('doubted', 'if')

# 50語のループを生成
for _ in range(50):
    # 現在のビグラムに基づいて後続の語のリストを取得
    successors = successor_map.get(bigram)
    
    # 後続の語が存在する場合、ランダムに選択して表示
    if successors:
        next_word = random.choice(successors)
        print(next_word)
        
        # 新しいビグラムを設定
        bigram = (bigram[1], next_word)
    else:
        # 後続の語がない場合はループを終了
        print("終了: 後続が見つかりません")
        break
```

このコードは、指定された方法で50語を生成します。適切なデータで`successor_map`を設定することを忘れないでください。

In [ ]:
# Solution goes here

すべてが正常に動作している場合、生成されたテキストは元のスタイルに認識できるほど類似しており、いくつかのフレーズは意味を成すはずです。ただし、テキストはあるトピックから別のトピックに移ることがあるかもしれません。

ボーナス演習として、前回の2つの演習の解決策を修正し、`successor_map` のキーとしてトライグラムを使用して、結果にどのような影響があるかを確認してください。

『Think Python: 第3版』(https://allendowney.github.io/ThinkPython/index.html)

著作権 2024 [Allen B. Downey](https://allendowney.com)

コードライセンス: [MITライセンス](https://mit-license.org/)

テキストライセンス: [クリエイティブ・コモンズ 表示-非営利-継承4.0国際](https://creativecommons.org/licenses/by-nc-sa/4.0/)